# Using MCP BRAVE Server for internet search with a Re-Act Agent

1. Import the necessary libraries

In [1]:
import asyncio
import logging
import os
import sys
import traceback
from typing import Any
from beeai_framework.agents.react.runners.default.prompts import SystemPromptTemplate
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from beeai_framework import Tool
from beeai_framework.agents.react.agent import ReActAgent
from beeai_framework.agents.types import AgentExecutionConfig
from beeai_framework.backend.chat import ChatModel, ChatModelParameters
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.errors import FrameworkError
from beeai_framework.logger import Logger
from beeai_framework.memory.token_memory import TokenMemory
from beeai_framework.tools.mcp_tools import MCPTool
from pathlib import Path
from beeai_framework.adapters.openai.backend.chat import OpenAIChatModel
from beeai_framework.backend.message import SystemMessage


2. Load the enviorment variables and set the system path (if needed)

In [2]:
import os
from dotenv import load_dotenv

# Absolute path to your .env file
# sometimes the system can have trouble locating the .env file
env_path = '/Users/sandi/beeai-framework/python/.env'
# Load it
load_dotenv(dotenv_path=env_path)

# Get current working directory
path = '/Users/sandi/beeai-framework/python'
# Append to sys.path
sys.path.append(path)



3. Configure the logger

In [3]:
# Configure logging - using DEBUG instead of trace
logger = Logger("app", level=logging.DEBUG)

4. Load helper functions like process_agent_events,observer, and create an instance of ConsoleReader.  
- process_agent_events: Handles agent events and logs messages to the console based on the event type (e.g., error, retry, update). It ensures meaningful output for each event to help track agent activity.
- observer: Listens for all events from an emitter and routes them to process_agent_events for processing and display.
- ConsoleReader: Manages console input/output, allowing user interaction and formatted message display with color-coded roles.

In [12]:
#load console reader
from examples.helpers.io import ConsoleReader
#this is a helper function that makes the assitant chat easier to read
reader = ConsoleReader()

def process_agent_events(data: dict[str, Any], event: EventMeta) -> None:
    """Process agent events and log appropriately"""

    if event.name == "error":
        reader.write("Agent 🤖 : ", FrameworkError.ensure(data["error"]).explain())
    elif event.name == "retry":
        reader.write("Agent 🤖 : ", "retrying the action...")
    elif event.name == "update":
        reader.write(f"Agent({data['update']['key']}) 🤖 : ", data["update"]["parsedValue"])
    elif event.name == "start":
        reader.write("Agent 🤖 : ", "starting new iteration")
    elif event.name == "success":
        reader.write("Agent 🤖 : ", "success")
    else:
        print(event.path)

def observer(emitter: Emitter) -> None:
    emitter.on("*.*", process_agent_events)


5. Set the Brave API Key and server parameters. Anthropic has a list of MCP servers here https://modelcontextprotocol.io/examples

In [6]:
brave_api_key = os.environ["BRAVE_API_KEY"]

brave_server_params = StdioServerParameters(
    command="/opt/homebrew/bin/npx",  # Full path to be safe
    args=[
        "-y",
        "@modelcontextprotocol/server-brave-search"
    ],
    env={
        "BRAVE_API_KEY": brave_api_key,
         "x-subscription-token": brave_api_key
    },
)

6. Create the brave tool that initatives the connection to the MCP server, discovers tools, and returns the discovered tools to the Agents so it can decide what tool is appropriate to call for a given task.  
In this case 2 tools are discoverable on the Brave MCP Server:
- brave_web_search: Execute web searches with pagination and filtering
- brave_local_search: Search for local businesses and services

In [7]:
async def brave_tool() -> MCPTool:
    brave_env = os.environ.copy()
    brave_server_params = StdioServerParameters(
        command="/opt/homebrew/bin/npx",
        args=["-y", "@modelcontextprotocol/server-brave-search"],
        env=brave_env
    )

    print("Starting MCP client...")
    try:
        async with stdio_client(brave_server_params) as (read, write), ClientSession(read, write) as session:
            print("Client connected, initializing...")

            await asyncio.wait_for(session.initialize(), timeout=10)
            print("Initialized! Discovering tools...")

            bravetools = await asyncio.wait_for(
                MCPTool.from_client(session, brave_server_params),
                timeout=10
            )
            print("Tools discovered!")
            return bravetools
    except asyncio.TimeoutError as e:
        print("❌ Timeout occurred during session initialization or tool discovery.")
    except Exception as e:
        print("❌ Exception occurred:", e)
        traceback.print_exc()


(Optional) Check the connection to the MCP server and ensure it returns all the available tools before providing it to the agent

In [9]:
tool = await brave_tool()
print("Discovered tools:", tool)

for tool in tool:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {getattr(tool, 'description', 'No description available')}")
    print("-" * 30)


Starting MCP client...
Client connected, initializing...
Initialized! Discovering tools...
Tools discovered!
Discovered tools: [<beeai_framework.tools.mcp_tools.MCPTool object at 0x119aa6c00>, <beeai_framework.tools.mcp_tools.MCPTool object at 0x10fee3e60>]
Tool Name: brave_web_search
Description: Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. 
------------------------------
Tool Name: brave_local_search
Description: Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:
- Business names and addresses
- Ratings and review counts
- Phone numbers and opening hours


7. Write the function that creates the agent:   
- assign an LLM
- create an instance of the brave_tool() function and assign it to a tools variable
- create a re-act agent and assign it the chosen llm, tools, memory (so it can have constinous conversation)
- Add a system prompt to the re-act agent.   

Note: You might notice that I added a sentence to the system prompt that reads "If you need to use the brave_tool you must use a count of 5." This is a bandaid work-around becasue of a bug I found in the index.ts file of the brave server. I will contribute to the repo to fix it.

In [ ]:
async def create_agent() -> ReActAgent:
    """Create and configure the agent with tools and LLM"""

    #using openai api instead
    llm = OpenAIChatModel(model_id="gpt-4o")
    
    # Configure tools
    tools: list[Tool] = await brave_tool()
    #tools: list[Tool] = [await brave_tool()]

    # Create agent with memory and tools
    agent = ReActAgent(llm=llm, tools=tools, memory=TokenMemory(llm), )
    # add system prompt (work-around for current Brave AMCP server bug)
    await agent.memory.add(SystemMessage(content="You are a helpful assistant. If you need to use the brave_tool you must use a count of 5."))

    return agent




8. Create the main function that:
- creates the agent
- enters a conversation loop with the user and running the agent with the user prompt and some configuration settings. Finishes the conversation if the user types "exit" or "quit".

In [13]:
import asyncio
import traceback
import sys

# Your async main function
async def main() -> None:
    """Main application loop"""

    # Create agent
    agent = await create_agent()

    # Main interaction loop with user input
    for prompt in reader:
        # Exit condition
        if prompt.strip().lower() in {"exit", "quit"}:
            reader.write("Session ended by user. Goodbye! 👋\n")
            break

        # Run agent with the prompt
        try:
            response = await agent.run(
                prompt=prompt,
                execution=AgentExecutionConfig(max_retries_per_step=3, total_max_retries=10, max_iterations=20),
            ).observe(observer)

            reader.write("Agent 🤖 : ", response.result.text)
        except Exception as e:
            reader.write("An error occurred: ", str(e))
            traceback.print_exc()


In [15]:
# Run main() with error handling
try:
    await main()  # Use await directly in Jupyter
except FrameworkError as e:
    traceback.print_exc()
    sys.exit(e.explain())

Starting MCP client...
Client connected, initializing...
Initialized! Discovering tools...
Tools discovered!
Interactive session has started. To escape, input 'q' and submit.
Agent 🤖 :  starting new iteration
Agent 🤖 :  starting new iteration
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
agent.react.partialUpdate
Agent(thought) 🤖 :  I will use the brave_local_search function to find the open hours for La Taqueria on Mi